# import packages

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
from scipy.signal import convolve
from mpl_toolkits.mplot3d import Axes3D
from numpy.fft import fftn, fftshift
%matplotlib widget

# import local files

In [ ]:
from ptable_dict import ptable, atomic_masses
from utilities import write_xyz, load_xyz, rotation_matrix, gaussian_kernel
from meshgrids import generate_density_grid, convert_grid_qspace, plot_3D_grid
from detector import make_detector, rotate_about_normal, rotate_about_horizontal, rotate_about_vertical, intersect_detector

# Generate and plot electron density voxel map for given xyz file

In [ ]:
dirr = '/Users/Thomas2/Library/CloudStorage/OneDrive-UCB-O365/Desktop/Research_Stuff/OPV_GIWAXS/forward_sim/'
xyz_path = f'{dirr}test_xyz_files/Benzene.xyz'
sigma = 1
voxel_size = 0.05
dens_grid, x_mesh, y_mesh, z_mesh = generate_density_grid(xyz_path, sigma, voxel_size)
threshold = 99.5
num_levels=5
cmap = 'plasma'
plot_3D_grid(dens_grid, x_mesh, y_mesh, z_mesh, cmap, threshold, num_levels)

# Generate and plot electron density reciprocal space voxel map for given xyz file

In [ ]:
iq, qx, qy, qz = convert_grid_qspace(dens_grid, x_mesh, y_mesh, z_mesh)
threshold = 99.8
num_levels=20
cmap = 'plasma'
plot_3D_grid(iq, qx, qy, qz, cmap, threshold, num_levels)

# find q-resolutions

In [ ]:
x_vals = qx[0,:,0]
y_vals = qy[:,0,0]
z_vals = qz[0,0,:]
qx_res = x_vals[1]-x_vals[0]
qy_res = y_vals[1]-y_vals[0]
qz_res = z_vals[1]-z_vals[0]
print(f'Resolutions are [qx={qx_res:.4f}, qy={qy_res:.4f}, qz={qz_res:.4f}]')

# Set up Detector

In [ ]:
det_pixels = (200,200) #horizontal, vertical
det_qs = (15,15) #horizontal, vertical (these are absolute maximums. detector centered at 0)
det_x_grid, det_y_grid, det_z_grid, det_h, det_v = make_detector(det_qs[0], det_pixels[0], det_qs[1], det_pixels[1])


# Rotate detector

In [ ]:
psi = 0 #rotation in degrees of detector about detector normal axis
det_x_grid, det_y_grid, det_z_grid = rotate_about_normal(det_x_grid, det_y_grid, det_z_grid, psi)

In [ ]:
phi = 90 #rotation in degrees of detector about detector vertical axis
det_x_grid, det_y_grid, det_z_grid = rotate_about_vertical(det_x_grid, det_y_grid, det_z_grid, phi)

In [ ]:
theta = 90 #rotation in degrees of detector about detector horizontal axis
det_x_grid, det_y_grid, det_z_grid = rotate_about_vertical(det_x_grid, det_y_grid, det_z_grid, theta)

In [ ]:
det_ints = intersect_detector(iq, qx, qy, qz, det_x_grid, det_y_grid, det_z_grid, det_h, det_v)

# plot
fig, ax1 = subplots()
ax1.imshow(det_ints,
           norm=matplotlib.colors.Normalize(),
           extent=(np.min(det_h),np.max(det_h),np.min(det_v),np.max(det_v)),
           cmap='turbo',
           origin = 'lower')
ax1.set_xlabel('qxy')
ax1.set_ylabel('qz')